In [10]:

# ============================================================
# Walmart Mini Dashboard (ALL graphs change by Store)
# Run: python walmart_mini_dashboard.py
# Open: http://127.0.0.1:8050
# ============================================================

import pandas as pd
import numpy as np
from threading import Timer
import webbrowser

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor

import plotly.graph_objects as go
import plotly.express as px

from dash import Dash, dcc, html, Input, Output


# ------------------------------------------------------------
# 1) Load + Feature Engineering
# ------------------------------------------------------------
df = pd.read_csv("Walmart.csv")

# Your file uses dd-mm-yyyy format
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, errors="coerce")
df = df.dropna(subset=["Date", "Weekly_Sales"]).copy()
df = df.sort_values(["Date", "Store"]).reset_index(drop=True)

# Simple time features
df["Year"]  = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Week"]  = df["Date"].dt.isocalendar().week.astype(int)

feature_cols = [
    "Store", "Holiday_Flag", "Temperature",
    "Fuel_Price", "CPI", "Unemployment",
    "Year", "Month", "Week"
]

X = df[feature_cols].copy()
y = df["Weekly_Sales"].copy()


# ------------------------------------------------------------
# 2) Train/Test Split (Time-based)
# ------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, shuffle=False
)
test_idx = X_test.index


# ------------------------------------------------------------
# 3) Train Model
# ------------------------------------------------------------
model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective="reg:squarederror",
    tree_method="hist"
)
model.fit(X_train, y_train)

pred = model.predict(X_test)

# global metrics (test)
rmse_global = mean_squared_error(y_test, pred) ** 0.5
mae_global  = mean_absolute_error(y_test, pred)


# ------------------------------------------------------------
# 4) Prediction Records (Aligned)
# ------------------------------------------------------------
res = df.loc[test_idx, ["Date", "Store", "Holiday_Flag", "Weekly_Sales"]].copy()
res["Predicted"] = pred
res["Residual"]  = res["Weekly_Sales"] - res["Predicted"]
res["Abs_Error"] = res["Residual"].abs()
res = res.sort_values(["Store", "Date"]).reset_index(drop=True)

# optional: save for your report
res.to_csv("prediction_records.csv", index=False)

stores = sorted(res["Store"].unique())


# ------------------------------------------------------------
# 5) Helpers: KPI + Figures (ALL depend on selected Store)
# ------------------------------------------------------------
def store_kpis(store_id: int):
    sd = res[res["Store"] == store_id]
    if len(sd) == 0:
        return 0, 0, 0, 0, 0

    rmse_s = (mean_squared_error(sd["Weekly_Sales"], sd["Predicted"]) ** 0.5)
    mae_s  = mean_absolute_error(sd["Weekly_Sales"], sd["Predicted"])
    avg_a  = sd["Weekly_Sales"].mean()
    avg_p  = sd["Predicted"].mean()
    n      = len(sd)
    return rmse_s, mae_s, avg_a, avg_p, n


def fig_store_line(store_id: int):
    sd = res[res["Store"] == store_id].sort_values("Date")

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=sd["Date"], y=sd["Weekly_Sales"],
        mode="lines", name="Actual", line=dict(width=3)
    ))
    fig.add_trace(go.Scatter(
        x=sd["Date"], y=sd["Predicted"],
        mode="lines", name="Predicted",
        line=dict(width=3, dash="dash")
    ))

    # Holiday markers
    hol = sd[sd["Holiday_Flag"] == 1]
    fig.add_trace(go.Scatter(
        x=hol["Date"], y=hol["Weekly_Sales"],
        mode="markers", name="Holiday",
        marker=dict(color="red", size=7)
    ))

    fig.update_layout(
        template="plotly_white",
        hovermode="x unified",
        title=f"Store {store_id}: Actual vs Predicted (Test) | Global RMSE={rmse_global:,.0f}",
        xaxis_title="Date",
        yaxis_title="Weekly Sales"
    )
    return fig


def fig_store_residual(store_id: int):
    sd = res[res["Store"] == store_id].sort_values("Date")

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=sd["Date"], y=sd["Residual"],
        mode="lines", name="Residual",
        line=dict(color="purple", width=2)
    ))
    fig.add_hline(y=0, line_width=1, line_color="black")

    fig.update_layout(
        template="plotly_white",
        hovermode="x unified",
        title=f"Store {store_id}: Residuals Over Time",
        xaxis_title="Date",
        yaxis_title="Actual - Predicted"
    )
    return fig


def fig_store_hist(store_id: int):
    sd = res[res["Store"] == store_id]

    fig = px.histogram(
        sd, x="Residual", nbins=35,
        template="plotly_white",
        title=f"Store {store_id}: Residual Distribution"
    )
    fig.update_layout(xaxis_title="Residual", yaxis_title="Count")
    return fig


def fig_store_scatter(store_id: int):
    sd = res[res["Store"] == store_id]

    fig = px.scatter(
        sd, x="Weekly_Sales", y="Predicted",
        color="Holiday_Flag",
        template="plotly_white",
        title=f"Store {store_id}: Actual vs Predicted (Scatter)",
        labels={"Weekly_Sales": "Actual", "Predicted": "Predicted"}
    )

    # Ideal y = x line
    minv = float(min(sd["Weekly_Sales"].min(), sd["Predicted"].min()))
    maxv = float(max(sd["Weekly_Sales"].max(), sd["Predicted"].max()))
    fig.add_trace(go.Scatter(
        x=[minv, maxv], y=[minv, maxv],
        mode="lines", name="Ideal (y=x)",
        line=dict(color="red", dash="dash")
    ))
    return fig


def fig_store_holiday_error(store_id: int):
    sd = res[res["Store"] == store_id].copy()
    sd["Abs_Error"] = sd["Residual"].abs()

    fig = px.box(
        sd, x="Holiday_Flag", y="Abs_Error",
        template="plotly_white",
        title=f"Store {store_id}: Abs Error (Holiday vs Non-Holiday)",
        labels={"Holiday_Flag": "Holiday (0=No, 1=Yes)", "Abs_Error": "Absolute Error"}
    )
    return fig


def kpi_cards(rmse_s, mae_s, avg_a, avg_p, n):
    box_style = {
        "padding": "10px",
        "border": "1px solid #ddd",
        "borderRadius": "8px",
        "width": "20%",
        "textAlign": "center"
    }
    return [
        html.Div([html.H4("Rows"), html.H3(f"{n}")], style=box_style),
        html.Div([html.H4("Store RMSE"), html.H3(f"{rmse_s:,.0f}")], style=box_style),
        html.Div([html.H4("Store MAE"), html.H3(f"{mae_s:,.0f}")], style=box_style),
        html.Div([html.H4("Avg Actual"), html.H3(f"{avg_a:,.0f}")], style=box_style),
        html.Div([html.H4("Avg Pred"), html.H3(f"{avg_p:,.0f}")], style=box_style),
    ]


# ------------------------------------------------------------
# 6) DASH APP (ALL graphs dynamic)
# ------------------------------------------------------------
app = Dash(__name__)
app.title = "Walmart Mini Dashboard"

app.layout = html.Div([
    html.H2("Walmart Sales Forecasting — Mini Dashboard (All Graphs Store-Controlled)"),

    html.Div([
        html.Label("Select Store"),
        dcc.Dropdown(
            id="store_dd",
            options=[{"label": f"Store {s}", "value": s} for s in stores],
            value=stores[0],
            clearable=False,
            style={"width": "250px"}
        ),
    ], style={"marginBottom": "10px"}),

    html.Div(id="kpi_row", style={"display": "flex", "gap": "10px", "marginBottom": "10px"}),

    dcc.Graph(id="g_store_line"),
    html.Div([
        html.Div([dcc.Graph(id="g_store_residual")], style={"width": "50%"}),
        html.Div([dcc.Graph(id="g_store_hist")], style={"width": "50%"}),
    ], style={"display": "flex", "gap": "10px"}),

    html.Div([
        html.Div([dcc.Graph(id="g_store_scatter")], style={"width": "50%"}),
        html.Div([dcc.Graph(id="g_store_holiday_err")], style={"width": "50%"}),
    ], style={"display": "flex", "gap": "10px"}),

], style={"maxWidth": "1200px", "margin": "0 auto", "fontFamily": "Arial"})


@app.callback(
    Output("kpi_row", "children"),
    Output("g_store_line", "figure"),
    Output("g_store_residual", "figure"),
    Output("g_store_hist", "figure"),
    Output("g_store_scatter", "figure"),
    Output("g_store_holiday_err", "figure"),
    Input("store_dd", "value")
)
def update_all(store_id):
    store_id = int(store_id)

    rmse_s, mae_s, avg_a, avg_p, n = store_kpis(store_id)

    return (
        kpi_cards(rmse_s, mae_s, avg_a, avg_p, n),
        fig_store_line(store_id),
        fig_store_residual(store_id),
        fig_store_hist(store_id),
        fig_store_scatter(store_id),
        fig_store_holiday_error(store_id)
    )


# ------------------------------------------------------------
# 7) RUN on 127.0.0.1:8050 (Dash new versions use app.run)
# ------------------------------------------------------------
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050")

if __name__ == "__main__":
    Timer(1, open_browser).start()
    app.run(host="127.0.0.1", port=8053, debug=True)


In [ ]:
)
model.fit(X_train, y_train)

pred = model.predict(X_test)

# global metrics (test)
rmse_global = mean_squared_error(y_test, pred) ** 0.5
mae_global  = mean_absolute_error(y_test, pred)


# ------------------------------------------------------------
# 4) Prediction Records (Aligned)
# ------------------------------------------------------------
res = df.loc[test_idx, ["Date", "Store", "Holiday_Flag", "Weekly_Sales"]].copy()
res["Predicted"] = pred
res["Residual"]  = res["Weekly_Sales"] - res["Predicted"]
res["Abs_Error"] = res["Residual"].abs()
res = res.sort_values(["Store", "Date"]).reset_index(drop=True)

# optional: save for your report
res.to_csv("prediction_records.csv", index=False)

stores = sorted(res["Store"].unique())


# ------------------------------------------------------------
# 5) Helpers: KPI + Figures (ALL depend on selected Store)
# ------------------------------------------------------------
def store_kpis(store_id: int):
    sd = res[res["Store"] == store_id]
    if len(sd) == 0:
        return 0, 0, 0, 0, 0

    rmse_s = (mean_squared_error(sd["Weekly_Sales"], sd["Predicted"]) ** 0.5)
    mae_s  = mean_absolute_error(sd["Weekly_Sales"], sd["Predicted"])
    avg_a  = sd["Weekly_Sales"].mean()
    avg_p  = sd["Predicted"].mean()
    n      = len(sd)
    return rmse_s, mae_s, avg_a, avg_p, n


def fig_store_line(store_id: int):
    sd = res[res["Store"] == store_id].sort_values("Date")

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=sd["Date"], y=sd["Weekly_Sales"],
        mode="lines", name="Actual", line=dict(width=3)
    ))
    fig.add_trace(go.Scatter(
        x=sd["Date"], y=sd["Predicted"],
        mode="lines", name="Predicted",
        line=dict(width=3, dash="dash")
    ))

    # Holiday markers
    hol = sd[sd["Holiday_Flag"] == 1]
    fig.add_trace(go.Scatter(
        x=hol["Date"], y=hol["Weekly_Sales"],
        mode="markers", name="Holiday",
        marker=dict(color="red", size=7)
    ))

    fig.update_layout(
        template="plotly_white",
        hovermode="x unified",
        title=f"Store {store_id}: Actual vs Predicted (Test) | Global RMSE={rmse_global:,.0f}",
        xaxis_title="Date",
        yaxis_title="Weekly Sales"
    )
    return fig


def fig_store_residual(store_id: int):
    sd = res[res["Store"] == store_id].sort_values("Date")

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=sd["Date"], y=sd["Residual"],
        mode="lines", name="Residual",
        line=dict(color="purple", width=2)
    ))
    fig.add_hline(y=0, line_width=1, line_color="black")

    fig.update_layout(
        template="plotly_white",
        hovermode="x unified",
        title=f"Store {store_id}: Residuals Over Time",
        xaxis_title="Date",
        yaxis_title="Actual - Predicted"
    )
    return fig


def fig_store_hist(store_id: int):
    sd = res[res["Store"] == store_id]

    fig = px.histogram(
        sd, x="Residual", nbins=35,
        template="plotly_white",
        title=f"Store {store_id}: Residual Distribution"
    )
    fig.update_layout(xaxis_title="Residual", yaxis_title="Count")
    return fig


def fig_store_scatter(store_id: int):
    sd = res[res["Store"] == store_id]

    fig = px.scatter(
        sd, x="Weekly_Sales", y="Predicted",
        color="Holiday_Flag",
        template="plotly_white",
        title=f"Store {store_id}: Actual vs Predicted (Scatter)",
        labels={"Weekly_Sales": "Actual", "Predicted": "Predicted"}
    )

    # Ideal y = x line
    minv = float(min(sd["Weekly_Sales"].min(), sd["Predicted"].min()))
    maxv = float(max(sd["Weekly_Sales"].max(), sd["Predicted"].max()))
    fig.add_trace(go.Scatter(
        x=[minv, maxv], y=[minv, maxv],
        mode="lines", name="Ideal (y=x)",
        line=dict(color="red", dash="dash")
    ))
    return fig


def fig_store_holiday_error(store_id: int):
    sd = res[res["Store"] == store_id].copy()
    sd["Abs_Error"] = sd["Residual"].abs()

    fig = px.box(
        sd, x="Holiday_Flag", y="Abs_Error",
        template="plotly_white",
        title=f"Store {store_id}: Abs Error (Holiday vs Non-Holiday)",
        labels={"Holiday_Flag": "Holiday (0=No, 1=Yes)", "Abs_Error": "Absolute Error"}
    )
    return fig


def kpi_cards(rmse_s, mae_s, avg_a, avg_p, n):
    box_style = {
        "padding": "10px",
        "border": "1px solid #ddd",
        "borderRadius": "8px",
        "width": "20%",
        "textAlign": "center"
    }
    return [
        html.Div([html.H4("Rows"), html.H3(f"{n}")], style=box_style),
        html.Div([html.H4("Store RMSE"), html.H3(f"{rmse_s:,.0f}")], style=box_style),
        html.Div([html.H4("Store MAE"), html.H3(f"{mae_s:,.0f}")], style=box_style),
        html.Div([html.H4("Avg Actual"), html.H3(f"{avg_a:,.0f}")], style=box_style),
        html.Div([html.H4("Avg Pred"), html.H3(f"{avg_p:,.0f}")], style=box_style),
    ]


# ------------------------------------------------------------
# 6) DASH APP (ALL graphs dynamic)
# ------------------------------------------------------------
app = Dash(__name__)
app.title = "Walmart Mini Dashboard"

app.layout = html.Div([
    html.H2("Walmart Sales Forecasting — Mini Dashboard (All Graphs Store-Controlled)"),

    html.Div([
        html.Label("Select Store"),
        dcc.Dropdown(
            id="store_dd",
            options=[{"label": f"Store {s}", "value": s} for s in stores],
            value=stores[0],
            clearable=False,
            style={"width": "250px"}
        ),
    ], style={"marginBottom": "10px"}),

    html.Div(id="kpi_row", style={"display": "flex", "gap": "10px", "marginBottom": "10px"}),

    dcc.Graph(id="g_store_line"),
    html.Div([
        html.Div([dcc.Graph(id="g_store_residual")], style={"width": "50%"}),
        html.Div([dcc.Graph(id="g_store_hist")], style={"width": "50%"}),
    ], style={"display": "flex", "gap": "10px"}),

    html.Div([
        html.Div([dcc.Graph(id="g_store_scatter")], style={"width": "50%"}),
        html.Div([dcc.Graph(id="g_store_holiday_err")], style={"width": "50%"}),
    ], style={"display": "flex", "gap": "10px"}),

], style={"maxWidth": "1200px", "margin": "0 auto", "fontFamily": "Arial"})


@app.callback(
    Output("kpi_row", "children"),
    Output("g_store_line", "figure"),
    Output("g_store_residual", "figure"),
    Output("g_store_hist", "figure"),
    Output("g_store_scatter", "figure"),
    Output("g_store_holiday_err", "figure"),
    Input("store_dd", "value")
)
def update_all(store_id):
    store_id = int(store_id)

    rmse_s, mae_s, avg_a, avg_p, n = store_kpis(store_id)

    return (
        kpi_cards(rmse_s, mae_s, avg_a, avg_p, n),
        fig_store_line(store_id),
        fig_store_residual(store_id),
        fig_store_hist(store_id),
        fig_store_scatter(store_id),
        fig_store_holiday_error(store_id)
    )


# ------------------------------------------------------------
# 7) RUN on 127.0.0.1:8050 (Dash new versions use app.run)
# ------------------------------------------------------------
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050")

if __name__ == "__main__":
    Timer(1, open_browser).start()
    app.run(host="127.0.0.1", port=8051, debug=True)
